# Divers  

In [ ]:
#bande de bolinger 

#utilitée : permet d avoir une idee de la volatilitée de al var (si grosse vol largeur ++; si petite vol largueur --)

#Sur la courbe il y a :
#    le mean
#    le mean + 2*std
#    le mean - 2*std


df['close 20day mean'] = df ['close'].rolling(20).mean()
df['Upper'] = df['close 20day mean'] + ( 2 * df ['close'].rolling(20).std() )
df['Lower'] = df['close 20day mean'] - ( 2 * df ['close'].rolling(20).std() )

In [ ]:
#petite astuce 
df.describe().transpose()

In [ ]:
#resample fonction d agregation temporelle 

df.resample ( rule = 'BQ')
# c est cool pour les tracer permet de grouper les données afin d avoir une vision sur l année ou sur le bq

In [ ]:
#Rendement :

#calcul de rendement simple : 
df['return'] = (  df['col'] /  df['col'].shift ( 1 ) ) - 1 
#rendement cumulatif :
df['return cumul' ] = ( 1 + df ['return' ] ).cumplod()


definition 

- tendance : (3 possibilitées)
a la hausse ,
stationnaire,
a la baisse ;
la tendance est sur le mean

- saisonalités : 
tendances qui se repete avec une regularité temporelle (la vente de pull en hiver)

- le cycle :
tendance avec des repetitions non definit


la variance : les dispertions des val de y dans le temps (dispertion de l echantillion)

la covariance : la vitesse de changement de la val


----

# Modele ETS

In [ ]:
# --------------------- Modele ETS : #error #tendance #saisonalités
#decompose la cource en 3 composantes 

from statsmodels.tsa.seasonal import seasonal_decompose

result = seasonall_decompose ( df[col] , model = x )  
     #x = 'multiplicative'  (si courbe semble non linaire)   
      ou = 'additive' (si courbe lineaire)

fig = result.plot()  


# Modele EWMA

In [ ]:
# --------------------- Modele EWMA : moyenne mobile pondere exponnentionellement

#La methode rolling permet de calculer la moyenne mobile MA ou SMA (simple moyenne average) mais a plusieurs desavantages
#La SMA est + utilisé pour  avoir une tendance. 
#La EWMA est bcp plus juste mathematiquement

df['EWMA12'] = df['Thousands of Passengers'].ewm(span=12).mean()  #12mois = 1an la saisonalite
df[['Thousands of Passengers','EWMA12']].plot()


# MODELE ARIMA

In [ ]:
# --------------------- #Modèle ARIMA  
#(AR :auto regressif ; I : (integrated) diff du Modele ARMA car serie stationnaire ; MA : moyenne mobile)

#les  ARIMA sont de 2 types:  saisonier et non saisonier


# import 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
%matplotlib inline
airpass = pd.read_csv('AirPassengersv2.csv', header=0, parse_dates=[0], index_col=0, squeeze=True)

#analyse 
#type (airpass) #type
airpass.plot()  #forme




#1 ---------------------------------------------------------- Verifier que la série temporelle est stationnaire, 
#plusieurs façons :

#methode 1 : analyse 
#Statistiques roulantes : Tracer la moyenne mobile et l’écart-type mobile. 
#La série temporelle est stationnaire si elle reste constante dans le temps 
#(à l’œil nu, regardez si les lignes sont droites et parallèles à l’axe des x)

type (airpass) #type
airpass.plot()  #forme
airpass.rolling(20).mean().plot()  #non // a x
airpass.rolling(20).std().plot()   #non // a x

from statsmodels.tsa.seasonal import seasonal_decompose   #decomposition ETS
result = seasonal_decompose ( airpass, model = 'multiplicative' )
fig = result.plot()



#methode 2 : Test de Dickey-Fuller augmenté (ADF) 
#La série temporelle est considérée comme stationnaire si la valeur p est < 0.05 (hypothèse nulle)

#Avec une fonction cree  :  
adf_check(airpasslog) #la fonction est ecrite apres cette celule



#Avec statsmodels directment : 

from statsmodels.tsa.stattools import adfuller

sm.tsa.stattools.adfuller(airpasslog_2)



#Lorsque la pvalue est < 0.005 cela  signifie que la courbe est stationnaire




#methode 3 : Cours datasientest

airpasslog  = np.log( airpass)  #liniarisation de la courbe 
pd.plotting.autocorrelation_plot(airpasslog) #Affiche la fonction d'autocorrélation de la série

#si retour du #2
airpasslog_1 = airpasslog.diff().dropna()
pd.plotting.autocorrelation_plot(airpasslog_1)



#la conclusion est que si la courbe n est pas stabilisée il faut passer a #2 si courbe stabilisée on passe a #3





#2 ---------------------------------------------------------- Differentiation - Rendre la time serie stable (si necessaire)
#plusieurs methodes

#Methode 1 : avec Pandas (sur une time serie ) - differnetiation : 
df[diff1] = diff[col] - diff[col].shift (1 )  
df[diff1].plot() # tracer la courbe diff1 pour avoir 1 apercu de la courbe si elle est stationnaire

df[season_diff1] = diff[col] - diff[col].shift (12 )  #saisonalite
df[season_diff1].plot() # tracer la courbe season_diff1 pour avoir 1 apercu de la courbe si elle est stationnaire

#analyse #1 Test de Dickey-Fuller augmenté (ADF)



#Methode 2 : avec Numpy (sur une time serie ) - differnetiation : 
airpasslog_1 = airpasslog.diff().dropna()  
df[airpasslog_1].plot()

airpasslog_2 = airpasslog_1.diff(periods = 12).dropna()   #saisonalite pour supprime les pics du a la saisonalite
df[airpasslog_2].plot()

#analyse #1 Test de Dickey-Fuller augmenté (ADF)
 
    

#conclusion : il faut absolument refaire le test Dickey-Fuller augmenté (ADF) sur la nouvelle colonne 
#afin de verifier que la pvalue est < 0.005 -> signifie que la courbe est stationnaire
#avec dropna () pour suppr les lignes perdues





#3 ----------------------------------------------------------  Choix des parametres  p, q et P, Q
#Choix des parametres  p d q du Modele de la Moyenne Mobile Auto-Régressive Intégrée (ARIMA)


from statsmodels.graphics.tsaplots import plot_pacf, plot_acf

plt.figure(figsize= (14,7))

plt.subplot(121)
#Fonction d’Auto-Corrélation (ACF) = PA = #p
plot_acf(airpasslog_2, lags = 36, ax=plt.gca())   

plt.subplot(122)
#Fonction d’Auto-Corrélation Partielle (PACF) = MA = #q 
plot_pacf(airpasslog_2, lags = 36, ax=plt.gca())  

plt.show()

# p = au terme p-1 avant qu il s annule ( zone de non significativité )
# q = au terme q-1 avant qu il s annule ( zone de non significativité )


#4 ----------------------------------------------------------  Construction du modele 

#attention airpasslog de base et le nombre D et d correspond a l ordre de la differentiation
#ARIMA = AR (auto regression P) ; I (integral D) ;MA (moyenne Q)

model=sm.tsa.SARIMAX(airpasslog,order=(1,1,1),seasonal_order=(1,1,1,12))   #modele Arima pour seasonal order 12 : season de 12
results=model.fit()       #fit
print(results.summary())  #resume sur le modele entrainé

#analyse du resumé
#modèle(données, order=(p,d,q), seasonal_order = (P,D,Q,s))


#5 ----------------------------------------------------------  Affinage
# attention on affine le modele . 
# le changement d un param entraine le changement de tous les param, 
#pour etre sur il faut a chaque changement recreer un model et le Fit 

#pour les param p,q :
#si regression liniaire suppr q
#si Moving average suppr p


#on recree le modele
model=sm.tsa.SARIMAX(airpasslog,order=(1,1,1),seasonal_order=(1,1,1,12))   #modele Arima pour seasonal order 12 : season de 12
results=model.fit()       #fit
print(results.summary())  #resume sur le modele entrainé


#pour les param P, Q :
#suppr par la suite les P ou Q d on la pValue >0.05


#on recree le modele avec les bon param
model=sm.tsa.SARIMAX(airpasslog,order=(1,1,1),seasonal_order=(1,1,1,12))   #modele Arima pour seasonal order 12 : season de 12
results=model.fit()       #fit
print(results.summary())  #resume sur le modele entrainé

#si c est ok la pvalue est proche ou egale a 0

#6 ----------------------------------------------------------  bruits blancs

#dans le resume : 

#Le test de Ljung-Box est un test de blancheur. 
#C'est un test statistique qui vise à rejeter ou non l'hypothèse  H0H0  : La résidu est un bruit blanc. 
#Ici on lit sur la ligne Prob(Q) que la p-valeur de ce test est de  0.650.65 , donc on ne rejette pas l'hypothèse.


#Le test de Jarque-Bera est un test de normalité. 
#C'est un test statistique qui vise à rejeter ou non l'hypothèse  H0H0  : Le résidu suit une distribution normale. 
#Ici on lit sur la ligne Prob (JB) que la p-valeur du test est de  0.720.72 . On ne rejette donc pas l'hypothèse.



#Conclusion
#Le résidu vérfie les hypothèses que l'on a faites à priori.   
#On peut donc conclure que le modèle  SARIMA(1,1,0)(0,1,1)12SARIMA⁡(1,1,0)(0,1,1)12  est satisfaisant.




#7 ----------------------------------------------------------  Prediction

pred = np.exp(results.predict(132, 143))   #la ligne 132 a 143 = 12 = 1an
airpasspred = pd.concat([airpass, pred])  #concat le reel et la predict
plt.plot(airpasspred)    #plot
plt.axvline(x='1960-01-01', color='red')   #train sur le plot entre le reel et la predict




#8 ---------------------------------------------------------- Verification Prediction / reel
airpass_test = pd.read_csv('AirPassengers.csv', header=0, parse_dates=[0], index_col=0, squeeze=True).iloc[132:] #import des lignes du jeu

plt.figure(figsize= (14,7))   #fig

plt.subplot(121)   #supplot 1
plt.plot(airpasspred);         #prediction
plt.plot(airpass_test, '--')   #reel

plt.subplot(122)   #supplot 2 : aux lignes 132 a 143
plt.plot(airpasspred.iloc[132:]);  #prediction
plt.plot(airpass_test, '--')      #reel

plt.show()




#9 ----------------------------------------------------------  évaluer l'erreur du modele MAPE

#Mean Average Prediction Error :  MAPE=1n∑t=1n||||Ytrue−YpredYtrue||||MAPE=1n∑t=1n|Ytrue−YpredYtrue|

pred = airpasspred.iloc[132:]
y_true, y_pred = np.array(airpass_test), np.array(pred)
MAPE = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
print("Mean Absolute Prediction Error : %0.2f%%"% MAPE)






In [ ]:
#fonction pour appliquer le Augmented Dickey-Fuller test sur une time serie

from statsmodels.tsa.stattools import adfuller

# Mémoriser dans une fonction pour une utilisation ultérieure !
def adf_check(time_series):
    """
    Passe une série temporelle, retourne le rapport ADF
    """
    result = adfuller(time_series)
    print('Augmented Dickey-Fuller test (DAF):')
    labels = ['ADF Test Statistic','p-value','#Lags Used','Number of Observations Used']

    for value,label in zip(result,labels):
        print(label+' : '+str(value) )
    
    if result[1] <= 0.05:
        print("Preuves solides contre l'hypothèse nulle, rejeter l'hypothèse nulle. Les données n'ont pas de racine unitaire et sont stationnaires")
    else:
        print("Faible preuve contre l'hypothèse nulle, la série temporelle a une racine unitaire, ce qui indique qu'elle est non stationnaire ")

Charger les bibliothèques pandas, numpy et statsmodels sous les noms pd, np et sm
Charger le package matplolib.pyplot sous le nom plt

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
%matplotlib inline


Chargez les données contenues dans le fichier portland_v2.csv
Attention au format de l'index et au type des données avant de passer aux questions suivantes

In [4]:
portland = pd.read_csv('portland_v2.csv',parse_dates=[0], index_col=0, squeeze=True)
portland.head(2)
portlandlog  = np.log( portland)

Représentez graphiquement les valeurs de la série

In [ ]:
portlandlog.plot();

Effectuez une décomposition saisonnière à l'aide de statsmodels
Quelle est la période ? Privilégiera-t-on un modèle additif ou multiplicatif ?

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose   #decomposition ETS
result = seasonal_decompose ( portlandlog, model = 'multiplicative' )
fig = result.plot()
#période = 12mois 
#le modele multiplicative est + adapté 

Affichez la fonction d'autocorrélation de la série

In [ ]:
pd.plotting.autocorrelation_plot(portlandlog)

Effectuez les opérations nécessaires pour estimer les paramètre  dd  et  DD  du modèle  SARIMASARIMA

In [ ]:
portland_1 = portlandlog.diff().dropna()  
pd.plotting.autocorrelation_plot(portland_1)

In [ ]:
 #saisonalite :
portland_2 = portland_1.diff(periods = 12).dropna()  
pd.plotting.autocorrelation_plot(portland_2)  #les pics sont atteinués et reste proche de 0

In [ ]:
from statsmodels.tsa.stattools import adfuller

sm.tsa.stattools.adfuller(portland_2)

#la pvalue est <0.05 -> la courbe est stationnaire cela est confirmé

À l'aide des fonction ACF et PACF, estimez les autres paramètres probables du modele  SARIMASARIMA

In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf

plt.figure(figsize= (14,7))

plt.subplot(121)
plot_acf(portland_2, lags = 36, ax=plt.gca())   

plt.subplot(122)
plot_pacf(portland_2, lags = 36, ax=plt.gca())  

plt.show()


Estimez à l'aide de la fonction SARIMAX le meilleur modèle possible pour cette série. Il se peut que tous les tests statistiques de normalité et de blancheur ne soient pas satisfaits.

In [ ]:
#Fonction d’Auto-Corrélation (ACF) = PA = #p s annule au rang 1  
#Fonction d’Auto-Corrélation Partielle (PACF) = MA = #q s annule au rang 1

model=sm.tsa.SARIMAX(portlandlog,order=(1,1,0),seasonal_order=(1,1,0,12))   
results=model.fit()       #fit
print(results.summary())  #resume sur le modele entrainé

# bruits blanc
#test de Ljung-Box val = 0.94 
#test de Jarque-Bera val = 0.82 

Effectuez une prédiction pour les 12 mois suivant la dernière mesure disponible

In [ ]:
# Insérez votre code ici
portland.describe()

pred = np.exp(results.predict(102, 113))   
portlandpred = pd.concat([portland, pred])  
plt.plot(portlandpred)    #
plt.axvline(x='1981-06-01', color='red')   


Lire les données présentes dans ce fichier
Faire une représentation graphique des valeurs prédites et des valeurs réelles de la série

In [2]:
# Insérez vote code ici 
portland_8182 = pd.read_csv('portland_8182.csv', header=0, parse_dates=[0], index_col=0,  squeeze=True)


plt.figure(figsize= (14,7))   

plt.subplot(121)   
plt.plot(portlandpred);         
plt.plot(portland_8182, '--')   

plt.subplot(122)   
plt.plot(pred);  
plt.plot(portland_8182, '--')     

plt.show()

NameError: name 'pd' is not defined

Calculez l'erreur moyenne relative de la prédiction.
Que concluez-vous ? Sous-évaluation ou sur-évaluation des valeurs ?

In [ ]:
pred = portlandpred.iloc[102:] [0]

y_true, y_pred = np.array(portland_8182), np.array(pred)

MAPE = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
print("Mean Absolute Prediction Error : %0.2f%%"% MAPE)

#la temporalité semble bonne mais l amplitutude de la courbe predite est trop petite 
# Sous-évaluation
